In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import feather
import gc
from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from numba import jit

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
%%time
data = pd.read_feather("finalData.ft")

Wall time: 8.57 s


In [3]:
%%time

features = data.columns.tolist()
features.remove('HasDetections')

labels = np.asarray(data['HasDetections'])
data.drop(['HasDetections'],inplace=True,axis=1)


data = np.asarray(data)


Wall time: 23.1 s


In [4]:
print(sum(np.isnan(data)))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [20]:
%%time
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.metrics import accuracy_score

folds = KFold(n_splits=3, shuffle=True)
scores=[]
confusionMatrices=[]
bestscore=0
for (train_index, test_index) in tqdm(folds.split(data,labels)):
    xtrain , ytrain = data[train_index] , labels[train_index]
    xtest , ytest = data[test_index] , labels[test_index]
    
    classifier = lgb.LGBMClassifier(boosting_type='gbdt', 
                                 num_leaves=250, 
                                 n_estimators=2000, 
                                 learning_rate=0.02,
                                   metric = 'binary_logloss')
    classifier.fit(xtrain , ytrain , eval_set=[(xtest, ytest)],verbose=200,early_stopping_rounds=100)
    pred = classifier.predict(xtest)
    score = roc_auc_score(ytest, pred)
    if(score>bestscore):
        bestscore=score
        joblib.dump(classifier, 'LightGBM.pkl')
    confusionMatrices.append(confusion_matrix(ytest,predicted))
    scores.append(score)
    
print (scores)




0it [00:00, ?it/s]

Training until validation scores don't improve for 100 rounds
[200]	valid_0's binary_logloss: 0.614865


KeyboardInterrupt: 